<h1><b>A. Nhập</b></h1>

<h3><b>1. Tải các thư viên</b></h3>

In [483]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 

<h3><b>2. Tải data

In [484]:
post_data = pd.read_csv('./dataset/csv_data/post_data.csv')
content_data = pd.read_csv('./dataset/csv_data/content_data.csv')
title_data = pd.read_csv('./dataset/csv_data/title_data.csv')

cleaned_data = pd.DataFrame() # Lưu kết quả cuối vào đây
# cleaned_data["url"] = post_data["url"]

cleaned_data['post_code'] = post_data['post_code']
cleaned_data['post_date'] = post_data['post_date']

cleaned_data["content"] = post_data["content"]
cleaned_data["title"] = post_data["title"]

cleaned_data["c_rent"] = content_data["rent"]
cleaned_data["t_rent"] = title_data["rent"]

cleaned_data["c_hurried"] = content_data["hurried"]
cleaned_data["t_hurried"] = title_data["hurried"]

cleaned_data["district"] = post_data["district"]

<h1><b>B. Replacing and Cleaning

Ở phần này, ta sẽ thay thế các data bị thiếu ở tập `main_data`, với thứ tự ưu tiên là `c_data` rồi `t_data`, vì các dữ liệu trong content sẽ được ghi chi tiết và đầy đủ hơn so với title.
Nếu như cả `c_data` và `t_data` đều trống, ta trả về `None` ở `main_data`

In [485]:
def replace_value(post, content, title):
    # Tạo màng gạt các giá trị null
    temp = content.copy()
    content_mask = content[(content.isnull()) | (content == 0)].index.values
    
    title = title.loc[content_mask]    
    title_mask = title[(title.isnull()) | (title == 0)].index.values

    title.loc[title_mask] = post.loc[title_mask]
    temp.loc[content_mask] = title.loc[content_mask]

    return temp

<h3><b>1. Price

Đối với giá tiền, trường hợp để trống dữ liệu sẽ không xảy ra, thay vào đó là "Thỏa thuận". Ngoài ra còn có các trường hợp nhập dư số không như 3899000000 tỷ hoặc 4500000 tỷ nhưng thực tế là 3.889 tỷ hoặc 4.5 tỷ

Giá trị của `price` nằm trong khoảng từ `1` đến nhỉnh hơn `1e+12`, trong đó từ giá trị từ `1` đến trước `200` sẽ được coi là chấp nhận được, các khoảng còn lại là bị lỗi nhập liệu. Cách sửa lỗi như sau:
- Từ `200` đến trước `1e6`: khoảng này rất đa dạng lỗi, nên sẽ không sửa và trả giá trị `None`
- Từ `1e6` đến trước `1e9`: khoảng nãy do người đăng nhập thêm 6 số 0, sửa bằng cách chia cho 1e6
- Từ `1e9` đến trước `1e12`: khoảng nãy do người đăng nhập thêm 9 số 0, sửa bằng cách chia cho 1e9
- `1e12` trở lên: khoảng nãy do người đăng nhập thêm 12 số 0, sửa bằng cách chia cho 1e12

In [486]:
def get_valid_price(price, thres = 350):
    if price != None:
        is_billions = re.search(r'(tỷ|tỉ)', str(price))        
        if not is_billions:
            has_number = re.search(r'\d+', str(price))
            if has_number:
                number = float(re.search(r'\d+', str(price)).group())
                if number > thres:
                    return price
                else:
                    return None
    return price

In [487]:
def clean_price(price, x=5):
    if type(price) == str:
        price = price.replace('giá','').replace('bán','').replace(':','')
        is_billions = re.search(r'(tỷ|tỉ)', price)
        
        if is_billions:
            normal_format = re.search(r'(\d+(\.\d+)?|\d+(\,\d+)?)', price)
            if normal_format:
                number = float(normal_format.group().replace(',', '.').replace('x', f'{x}'))
                # xử lý các trường hợp nhập dư số không
                if number > 1e+12:
                    number = number / 1e+12
                elif number < 1e+12 and number >= 1e+9:
                    number = number / 1e+9
                elif number < 1e+9 and number >= 1e+6:
                    number = number / 1e+6
                # elif number >= 200 and number < 1e+6:
                #     number = None
                return number
            else:
                number = float(price.replace(' ','').replace('tỷ','.').replace('tỉ','.'))
                return number

        elif not is_billions:
            is_millions = re.search(r'triệu', price)
            if is_millions:
                number = float(price.replace('triệu', '.').replace(' ', ''))
                return number / 1e+3

    return np.nan

In [488]:
post_price = post_data['price'].apply(get_valid_price)
content_price = content_data['price'].apply(get_valid_price)
title_price = title_data['price'].apply(get_valid_price)

cleaned_data["price"] = replace_value(post_price, content_price, title_price)
# cleaned_data["price"] = cleaned_data["price"].apply(clean_price)

In [489]:
price_1 = replace_value(post_price, content_price, title_price).apply(clean_price)
price_2 = replace_value(title_price, content_price, post_price).apply(clean_price)

index = []
true_price = []
for i in range(len(price_1)):
    if price_1[i] != price_2[i] and not np.isnan(price_1[i]) and not np.isnan(price_2[i]):
        if price_2[i] > 40:
            true_price.append(price_1[i])
        else:
            true_price.append(price_2[i])
        index.append(i)

true_price = pd.Series(true_price, index=index)
cleaned_data["price"].loc[index] = true_price.loc[index]
# cleaned_data["price"] = cleaned_data["price"].apply(clean_price)

C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1806904721.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  cleaned_data["price"].loc[index] = true_price.loc[index]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1806904721.py:15: Sett

<h3><b>2. Square</h3>

In [490]:
post_square = post_data['square']
content_square = content_data['square']
title_square = title_data['square']

cleaned_data["square"] = replace_value(post_square, content_square, title_square)

<h3><b>3. Bedroom </h3>

In [491]:
post_bedroom = post_data['bedroom']
content_bedroom = content_data['bedrooms']
title_bedroom = title_data['bedrooms']

cleaned_data["bedroom"] = replace_value(post_bedroom, content_bedroom, title_bedroom)

<h3><b>4. Bathroom</h3>

In [492]:
post_bathroom = post_data['bathroom']
content_bathroom = content_data['bathrooms']
title_bathroom = title_data['bathrooms']

cleaned_data["bathroom"] = replace_value(post_bathroom, content_bathroom, title_bathroom)

<h3><b>5. House direction</h3>

In [493]:
post_house_direction = post_data['house_direction']
content_house_direction = content_data['house_direction']
title_house_direction = title_data['house_direction']

post_house_direction = post_house_direction.apply(lambda x: x.lower() if x is not np.nan else x)
cleaned_data["house_direction"] = replace_value(post_house_direction, content_house_direction, title_house_direction)

<h3><b>6. Balcony</h3>

In [494]:
post_balcony = post_data['balcony']
content_balcony = content_data['balcony']
title_balcony = title_data['balcony']

cleaned_data["has_balcony"] = replace_value(post_balcony, content_balcony, title_balcony)
cleaned_data["has_balcony"].replace({np.nan: False, "Yes": True}, inplace=True )

C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\4155540165.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work becau

<h3><b>7. Furniture</h3>

In [495]:
post_furniture = post_data['furniture']
content_furniture = content_data['furniture']
title_furniture = title_data['furniture']

cleaned_data["has_furniture"] = replace_value(post_furniture, content_furniture, title_furniture)
cleaned_data["has_furniture"].replace({np.nan: False}, inplace=True)

C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\3146817283.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work becau

<h3><b>8. Legal information</h3>

In [496]:
post_legal_info = post_data['legal_info']
content_legal_info = content_data['legal_info']
title_legal_info = title_data['legal_info']

cleaned_data["has_legal_info"] = replace_value(post_legal_info, content_legal_info, title_legal_info)
cleaned_data["has_legal_info"].replace({np.nan: False}, inplace=True)

C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  title.loc[title_mask] = post.loc[title_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\1185534400.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  temp.loc[content_mask] = title.loc[content_mask]
C:\Users\dangm\AppData\Local\Temp\ipykernel_3024\206263825.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work becaus

<h3><b>9. Hurried</h3>

In [497]:
is_hurried_mask = cleaned_data[(cleaned_data["c_hurried"] == True) | (cleaned_data["t_hurried"] == True)].index.values
cleaned_data["is_hurried"] = False
cleaned_data.loc[is_hurried_mask, "is_hurried"] = True

<h3><b>10. Floors</h3>

In [498]:
post_floor = title_data['floors'].copy()
content_floor = content_data['floors'].copy()
title_floor = title_data['floors'].copy()

cleaned_data["floor"] = replace_value(post_floor, content_floor, title_floor)

<h3><b>11. Basements</h3>

In [499]:
post_basement = title_data['basement'].copy()
content_basement = content_data['basement'].copy()
title_basement = title_data['basement'].copy()

cleaned_data["has_basement"] = replace_value(post_basement, content_basement, title_basement)

<h3><b>12. House type</h3>

In [500]:
cleaned_data["house_type"] = post_data["type"].apply(lambda x: x.lower() if x is not np.nan else x)
cleaned_data["house_type"] = cleaned_data["house_type"].apply(lambda x: "nhà riêng" if x == "nhà" else x)

In [501]:
cleaned_data.drop(columns=["c_hurried", "t_hurried"], inplace=True)

In [503]:
price = cleaned_data['price']
cleaned_data.drop('price', axis=1, inplace=True)
cleaned_data['price'] = price

cleaned_data.to_csv('./dataset/csv_data/house_data.csv', index=False, encoding="utf-8-sig")
print("Done!")

Done!
